In [2]:
#standard message passing a heterogeneous graph
#gcn convultions
#gat mechanism

In [55]:
import csv
from torch_geometric.data import HeteroData
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling, train_test_split_edges
from torch.optim import Adam
from tqdm import tqdm
import random
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, GCNConv, to_hetero, HANConv, Linear
import torch.optim as optim
from sklearn.metrics import roc_auc_score

In [42]:
from torch_scatter import scatter

In [2]:
def preprocess_name(name):
    return re.split(r'[/#]', name)[-1]
def read_triples(file_path):
    triples = []

    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        next(reader)
        for row in reader:
            if len(row) == 3:
                triples.append(tuple(preprocess_name(col) for col in row))

    return triples

In [25]:
file_path = 'data/data_musee/sub_graph.tsv'

In [3]:
data = HeteroData()

In [27]:
# triples = read_triples(file_path)
# entities = set()
# relations = set()

# for subj, pred, obj in triples:
#     entities.add(subj)
#     entities.add(obj)
#     relations.add(pred)

# entity_idx = {name: idx for idx, name in enumerate(entities)}

# data['entity'].x = torch.arange(len(entities)).unsqueeze(1).float()

# torch.save(data['entity'].x, 'PyG_data/entity_features.pt')

# edges = {relation: [] for relation in relations}  # Edge containers for each relation

# for subj, pred, obj in triples:
#     edges[pred].append((entity_idx[subj], entity_idx[obj]))

# # Save edge index tensors using relation indices
# relation_mapping = {relation: idx for idx, relation in enumerate(relations)}

# for relation, edge_list in edges.items():
#     edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
#     relation_index = relation_mapping[relation]
#     torch.save(edge_index, f'PyG_data/direct_props/relation_{relation_index}_edge_index.pt')

# with open('PyG_data/relation_mapping.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['relation', 'index'])
#     for relation, index in relation_mapping.items():
#         writer.writerow([relation, index])

In [4]:
# loading KG into a Heterogeneous graph

data['entity'].x = torch.load('PyG_data/entity_features.pt')


relation_mapping = {}
with open('PyG_data/relation_mapping.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        relation, index = row
        
        relation_mapping[relation] = int(index)

# Load edge index tensors
for relation, index in relation_mapping.items():
    edge_index = torch.load(f'PyG_data/direct_props/relation_{index}_edge_index.pt')
    data[('entity', relation, 'entity')].edge_index = edge_index




In [5]:
data

HeteroData(
  entity={ x=[32153, 1] },
  (entity, P1412, entity)={ edge_index=[2, 6] },
  (entity, P6379, entity)={ edge_index=[2, 12] },
  (entity, P186, entity)={ edge_index=[2, 31457] },
  (entity, P119, entity)={ edge_index=[2, 2] },
  (entity, P361, entity)={ edge_index=[2, 328] },
  (entity, P156, entity)={ edge_index=[2, 17] },
  (entity, P195, entity)={ edge_index=[2, 19936] },
  (entity, P138, entity)={ edge_index=[2, 5] },
  (entity, P69, entity)={ edge_index=[2, 1] },
  (entity, P4969, entity)={ edge_index=[2, 106] },
  (entity, P840, entity)={ edge_index=[2, 7] },
  (entity, P1066, entity)={ edge_index=[2, 8] },
  (entity, P410, entity)={ edge_index=[2, 3] },
  (entity, P61, entity)={ edge_index=[2, 135] },
  (entity, P1196, entity)={ edge_index=[2, 1] },
  (entity, P737, entity)={ edge_index=[2, 7] },
  (entity, P1433, entity)={ edge_index=[2, 58] },
  (entity, P1830, entity)={ edge_index=[2, 1] },
  (entity, P562, entity)={ edge_index=[2, 44] },
  (entity, P1416, entity)=

In [22]:
def sample_data(data, sample_ratio=0.1):

    sampled_data = HeteroData()
    sampled_data['entity'].x = data['entity'].x

    for edge_type in data.edge_types:
        edge_index = data[edge_type].edge_index
        num_edges = edge_index.size(1)
        sample_size = int(num_edges * sample_ratio)
        perm = torch.randperm(num_edges)
        sampled_indices = perm[:sample_size]
        sampled_edge_index = edge_index[:, sampled_indices]
        sampled_data[edge_type].edge_index = sampled_edge_index

    return sampled_data

In [23]:
sample_ratio = 0.1
sampled_data = sample_data(data, sample_ratio)

In [24]:
# print("Edge types before processing:")
# for edge_type in data.edge_types:
#     print(edge_type)

In [25]:
def split_edges(edge_index, val_ratio=0.1):

    num_edges = edge_index.size(1)
    perm = torch.randperm(num_edges)
    val_size = int(num_edges * val_ratio)
    val_indices = perm[:val_size]
    train_indices = perm[val_size:]
    train_edge_index = edge_index[:, train_indices]
    val_edge_index = edge_index[:, val_indices]
    return train_edge_index, val_edge_index



In [26]:
train_data = HeteroData()
val_data = HeteroData()

train_data['entity'].x = sampled_data['entity'].x
val_data['entity'].x = sampled_data['entity'].x

for edge_type in sampled_data.edge_types:
    edge_index = sampled_data[edge_type].edge_index
    train_edge_index, val_edge_index = split_edges(edge_index)
    train_data[edge_type].edge_index = train_edge_index
    val_data[edge_type].edge_index = val_edge_index


In [27]:
def generate_negative_samples(data, num_negatives=1):
    negative_edges = {}
    for edge_type in data.edge_types:
        source_nodes = data[edge_type].edge_index[0]
        target_nodes = data[edge_type].edge_index[1]
        num_edges = source_nodes.size(0)
        
        negative_sources = source_nodes.repeat(num_negatives)
        negative_targets = torch.randint(0, data['entity'].x.size(0), (num_edges * num_negatives,))
        
        # Ensure that we are not sampling true edges as negatives
        negative_edges[edge_type] = torch.stack([negative_sources, negative_targets], dim=0)
        
    return negative_edges


In [28]:
def create_training_data(data, negative_edges):
    labels = {}
    for edge_type in data.edge_types:
        pos_edge_index = data[edge_type].edge_index
        neg_edge_index = negative_edges[edge_type]
        
        # Concatenate positive and negative edges
        all_edges = torch.cat([pos_edge_index, neg_edge_index], dim=1)
        
        # Create labels: 1 for positive edges, 0 for negative edges
        all_labels = torch.cat([torch.ones(pos_edge_index.size(1)), torch.zeros(neg_edge_index.size(1))])
        
        labels[edge_type] = all_labels
        
    return labels

In [48]:
!pip list | grep torch


torch                              1.11.0
torch-cluster                      1.6.3+pt21cu118
torch_geometric                    2.5.3
torch-max-mem                      0.1.3
torch-ppr                          0.0.8
torch-scatter                      2.0.9
torchaudio                         2.3.1+cu118
torchdata                          0.7.1
torchvision                        0.18.1+cu118


In [56]:
class HAN(nn.Module):
    def __init__(self, metadata, in_channels, hidden_channels, out_channels, heads=8):
        super(HAN, self).__init__()
        
        # Define the HANConv layers
        self.han_conv1 = HANConv(in_channels, hidden_channels, metadata, heads=heads)
        self.han_conv2 = HANConv(hidden_channels, out_channels, metadata, heads=heads)
        
        # Final classification layer
        self.lin = Linear(out_channels * heads, out_channels)
        
    def forward(self, x_dict, edge_index_dict):
        # Pass through the first HANConv layer
        x_dict = self.han_conv1(x_dict, edge_index_dict)
        x_dict = {key: F.elu(x) for key, x in x_dict.items()}
        
        # Pass through the second HANConv layer
        x_dict = self.han_conv2(x_dict, edge_index_dict)
        x_dict = {key: F.elu(x) for key, x in x_dict.items()}
        
        # Focus on the 'entity' node type for the final layer
        x = x_dict['entity']
        out = self.lin(x)
        
        return out

In [57]:
metadata = data.metadata()
metadata
in_channels = data['entity'].x.size(1)  # Get the number of input features from your entity data
model = HAN(metadata, in_channels=in_channels, hidden_channels=64, out_channels=32, heads=8)


1. built -in conversion from homogeneous GNN model to a heterogeneous GNN model


In [34]:
class GNN(torch.nn.Module):
  def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels, aggr='mean')
        self.conv2 = GCNConv(hidden_channels, out_channels, aggr='mean')

  def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x




aggregation  : mean (default)\
message : $M_{i,j} = \frac{x_j \cdot W}{\sqrt{D_i \cdot D_j}}$ \
update : ReLU



In [35]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super(LinkPredictor, self).__init__()
        self.linear = torch.nn.Linear(2 * in_channels, 1)

    def forward(self, x_i, x_j):
        x = torch.cat([x_i, x_j], dim=-1) # Concatenate node embeddings and apply linear layer
        return self.linear(x).squeeze(-1)




In [36]:
# data.metadata()

In [37]:
#generate negative samples for each edge type

def generate_negatives(data, edge_type, num_neg_samples=None):
    # if 'edge_index' not in data[edge_type]:
    #     raise AttributeError(f"Edge type {edge_type} has no edge_index attribute")

    edge_index = data[edge_type].edge_index.cpu()
    num_nodes = data['entity'].num_nodes
    neg_edge_index = negative_sampling(
        edge_index=edge_index,
        num_nodes=num_nodes,
        num_neg_samples=num_neg_samples,
        method='sparse'
    )
    return neg_edge_index



In [38]:
def get_train_data(data, negative_samples):
    pos_edge_index_dict = {}
    neg_edge_index_dict = {}

    for edge_type in data.edge_types:

        pos_edge_index_dict[edge_type] = data[edge_type].edge_index
        neg_edge_index_dict[edge_type] = negative_samples[edge_type]

    return pos_edge_index_dict, neg_edge_index_dict

In [39]:
#training loop

def train(model, predictor, data_loader, pos_edge_index_dict, neg_edge_index_dict):
    model.train()
    predictor.train()

    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        optimizer.zero_grad()

        x_dict = model(batch.x_dict, batch.edge_index_dict) #forward pass to update node emmbedding ,processes each edge type separately (conversion)
        batch_loss = 0
        for edge_type in batch.edge_types:
            pos_edge_index = pos_edge_index_dict[edge_type].long()
            neg_edge_index = neg_edge_index_dict[edge_type].long()

            pos_i = x_dict[edge_type[0]][pos_edge_index[0]] #source nodes
            pos_j = x_dict[edge_type[2]][pos_edge_index[1]] #target node

            neg_i = x_dict[edge_type[0]][neg_edge_index[0]]
            neg_j = x_dict[edge_type[2]][neg_edge_index[1]]

            pos_pred = predictor(pos_i, pos_j)
            neg_pred = predictor(neg_i, neg_j)

            pos_label = torch.ones(pos_pred.size(), device=device)
            neg_label = torch.zeros(neg_pred.size(), device=device)

            loss = criterion(pos_pred, pos_label) + criterion(neg_pred, neg_label)
            batch_loss += loss
            
        
        batch_loss.backward()
        optimizer.step()

        
        total_loss += batch_loss.item()

    
        torch.cuda.empty_cache()

    return total_loss / len(data_loader)

In [40]:
in_channels = data['entity'].x.size(1)
hidden_channels = 64
out_channels = 32

model = GNN(in_channels, hidden_channels, out_channels)
model = to_hetero(model, data.metadata(), aggr='mean')
predictor = LinkPredictor(out_channels)


In [41]:
negative_samples = {}
for edge_type in train_data.edge_types:
    neg_edge_index = generate_negatives(train_data, edge_type)
    negative_samples[edge_type] = neg_edge_index

pos_edge_index_dict, neg_edge_index_dict = get_train_data(train_data, negative_samples)


In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print("GPU Name:", torch.cuda.get_device_name(i))


GPU Name: NVIDIA A40
GPU Name: NVIDIA A40
GPU Name: NVIDIA A40


In [45]:

model = model.to(device)
predictor = predictor.to(device)
train_data = train_data.to(device)
val_data = val_data.to(device)


batch_size = 2
data_loader = DataLoader([train_data], batch_size=batch_size, shuffle=True)

# Training parameters
optimizer = Adam(list(model.parameters()) + list(predictor.parameters()), lr=0.0001)
criterion = BCEWithLogitsLoss()
num_epochs = 10


In [ ]:
for epoch in range(num_epochs):
    loss = train(model, predictor, data_loader, pos_edge_index_dict, neg_edge_index_dict)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

### Heterogeneous Convolution Wrapper
* define indivudual functions for each type